- (Gaussian) Multinomial Naive Bayes with Grid Search
- Logistic Regression
- Support Vector Machines
- Decision Tree
- Random Forest & Bagging Classifier
- Boosting XGBoost
- Neural Nets



In [33]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import sklearn.datasets as datasets
from sklearn.kernel_approximation import Nystroem
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from collections import Counter
from imblearn.over_sampling import SMOTE

In [21]:
# from LanguageModels.Word2Vec import Word2Vec
# from Preprocessing.LemmatizerPreprocessor import LemmatizerPreprocessor
# from Preprocessing.DataLoader import DataLoader

### Load Dataset

In [22]:
file = open("../data/processed/data.p",'rb')
data = pickle.load(file)
file.close()

file = open("../data/processed/labels.p",'rb')
labels = pickle.load(file)
file.close()

In [23]:
label = labels[:,0]
print('embeddings: {}'.format(data.shape))
print('label: {}'.format(label.shape))

embeddings: (1450, 200)
label: (1450,)


In [24]:
# get rid of Nones
# for i in range(label.shape[0]):
#     print(label[i])
#     if label[i] == None:
#         print(i)
# 299 -> 448

In [25]:
label = np.concatenate((label[:298], label[449::]), 0)
label=label.astype('int')
data = np.concatenate((data[:298], data[449::]), 0)
print('embeddings: {}'.format(data.shape))
print('label: {}'.format(label.shape))

embeddings: (1299, 200)
label: (1299,)


In [26]:
def f(i):
    if i == 11:
        return 0
    else:
        return 1
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42, stratify=label)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(Counter(y_train.tolist()))
print(Counter(y_test.tolist()))

##
y_train_binary = np.array([f(i) for i in y_train])
y_test_binary = np.array([f(i) for i in y_test])
##

(1039, 200) (260, 200) (1039,) (260,)
Counter({11: 560, 2: 113, 4: 86, 1: 54, 10: 52, 8: 47, 5: 44, 6: 36, 3: 23, 7: 18, 9: 6})
Counter({11: 140, 2: 28, 4: 22, 10: 13, 1: 13, 8: 12, 5: 11, 6: 9, 3: 6, 7: 4, 9: 2})


### Support Vector Machine: Multi-Class Classification

In [97]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})

### Test on Training Data

In [98]:
pred_train = clf.predict(X_train)
accuracy_score(y_train, pred_train)

0.9938311688311688

In [99]:
confusion_matrix(y_train, pred_train)

array([[560,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  2, 554,   0,   2,   1,   0,   0,   0,   0,   0,   1],
       [  0,   0, 560,   0,   0,   0,   0,   0,   0,   0,   0],
       [  1,   2,   2, 553,   0,   0,   0,   1,   0,   0,   1],
       [  0,   0,   0,   0, 560,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 560,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 560,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0, 560,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 560,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 560,   0],
       [  4,   6,   0,   9,   2,   0,   0,   1,   0,   3, 535]],
      dtype=int64)

### Test on Testing Data

In [100]:
pred_test = clf.predict(X_test)
accuracy_score(y_test, pred_test)

0.5884615384615385

In [101]:
confusion_matrix(y_test, pred_test)

array([[  6,   1,   0,   1,   0,   1,   1,   2,   0,   1,   0],
       [  4,  16,   1,   0,   0,   0,   0,   1,   0,   2,   4],
       [  0,   1,   0,   1,   2,   0,   0,   1,   0,   0,   1],
       [  4,   4,   0,   3,   1,   4,   0,   4,   0,   0,   2],
       [  0,   2,   0,   3,   5,   0,   0,   0,   0,   1,   0],
       [  3,   2,   1,   0,   1,   0,   1,   0,   0,   1,   0],
       [  1,   0,   1,   1,   0,   0,   0,   0,   0,   0,   1],
       [  5,   3,   0,   0,   1,   2,   0,   0,   0,   0,   1],
       [  0,   1,   0,   0,   1,   0,   0,   0,   0,   0,   0],
       [  2,   3,   0,   1,   1,   2,   0,   0,   0,   1,   3],
       [  0,   1,   2,   6,   1,   3,   1,   1,   0,   3, 122]],
      dtype=int64)

## Linear Regression & Evaluation of Different Types of Regularization

## Vanilla Linear Regression (No Regularization)

In [88]:
reg = LinearRegression().fit(X_train, y_train)
print("Training Accuracy: " + str(reg.score(X_train, y_train)))
print("Training Accuracy: " + str(reg.score(X_test, y_test)))
print(Counter(y_train.tolist()))

Training Accuracy: 0.6195824242931637
Training Accuracy: 0.4464623202341178
Counter({11: 560, 2: 113, 4: 86, 1: 54, 10: 52, 8: 47, 5: 44, 6: 36, 3: 23, 7: 18, 9: 6})


## Ridge Regression (L2 Regularization)

In [86]:
parameters = {'alpha':[1e-4, 1e-3, 1e-2, 0.1, 0.2, 0.5, 1, 2, 5, 10]}
rid = Ridge()
clf = GridSearchCV(rid, parameters, cv=10, verbose=1)
clf.fit(X_train, y_train)
print("Training Accuracy: " + str(clf.score(X_train, y_train)))
print("Testing Accuracy: " + str(clf.score(X_test, y_test)))
print(clf.best_params_)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Training Accuracy: 0.5454773677585926
Testing Accuracy: 0.5169477873199229
{'alpha': 2}


## Lasso Regression (L1 Regularization)

In [17]:
parameters = {'alpha':[1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0.2, 0.5, 1, 2, 5, 10, 100]}

las = Lasso(max_iter = 10000)
clf = GridSearchCV(las, parameters, cv=10, verbose=1)
clf.fit(X_train, y_train)
print("Training Accuracy: " + str(clf.score(X_train, y_train)))
print("Testing Accuracy: " + str(clf.score(X_test, y_test)))
print(clf.best_params_)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Training Accuracy: 0.5869160640569879
Testing Accuracy: 0.49856041968169207
{'alpha': 0.001}


## ElasticNet Regression (L1 + L2 Regularization)

In [18]:
parameters = {'alpha':[1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0.2, 0.5, 1, 2, 5, 10, 100], 'l1_ratio':[0.05, 0.1, 0.15, 0.25, 0.5, 0.75, 1]}

las = ElasticNet(max_iter = 10000)
clf = GridSearchCV(las, parameters, cv=10, verbose=1)
clf.fit(X_train, y_train)
print("Training Accuracy: " + str(clf.score(X_train, y_train)))
print("Testing Accuracy: " + str(clf.score(X_test, y_test)))
print(clf.best_params_)

Fitting 10 folds for each of 84 candidates, totalling 840 fits
Training Accuracy: 0.5675894103664105
Testing Accuracy: 0.520764478468223
{'alpha': 0.001, 'l1_ratio': 0.05}


## Train/Test on Multiple Classifiers

In [19]:
names = ["Linear SVM", "RBF SVM", "Gaussian Process", "Logistic Regression"
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "Bagging Classifier"]
         
classifiers = [
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    LogisticRegression(random_state=0, max_iter = 5000),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier(alpha=1, max_iter=5000),
    AdaBoostClassifier(),
    GaussianNB(),
    BaggingClassifier(base_estimator=SVC(),
                        n_estimators=10, random_state=0)]

score_dict = dict()
for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        score_dict[name] = score
        print(classification_report(clf.predict(X_test), y_test))
print(score_dict)

c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       1.00      0.54      0.70       260

    accuracy                           0.54       260
   macro avg       0.09      0.05      0.06       260
weighted avg       1.00      0.54      0.70       260



c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

              precision    recall  f1-score   support

           1       0.38      0.62      0.48         8
           2       0.68      0.35      0.46        54
           3       0.00      0.00      0.00         0
           4       0.14      0.21      0.17        14
           5       0.27      0.50      0.35         6
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.08      1.00      0.15         1
           9       0.00      0.00      0.00         0
          10       0.08      1.00      0.14         1
          11       0.97      0.77      0.86       176

    accuracy                           0.65       260
   macro avg       0.24      0.41      0.24       260
weighted avg       0.82      0.65      0.71       260



c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

...

c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

              precision    recall  f1-score   support

           1       0.08      1.00      0.14         1
           2       0.25      0.29      0.27        24
           3       0.00      0.00      0.00         0
           4       0.05      0.12      0.07         8
           5       0.18      1.00      0.31         2
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.99      0.61      0.76       225

    accuracy                           0.57       260
   macro avg       0.14      0.28      0.14       260
weighted avg       0.88      0.57      0.68       260



c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.57      0.23      0.33        70
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.97      0.72      0.83       188

    accuracy                           0.58       260
   macro avg       0.14      0.09      0.11       260
weighted avg       0.86      0.58      0.69       260

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00 

c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\cuong\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [20]:
names = ["Linear SVM", "RBF SVM", "Gaussian Process", "Logistic Regression"
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "Bagging Classifier"]
         
classifiers = [
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    LogisticRegression(random_state=0, max_iter = 5000),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier(alpha=1, max_iter=5000),
    AdaBoostClassifier(),
    GaussianNB(),
    BaggingClassifier(base_estimator=SVC(),
                        n_estimators=10, random_state=0)]

score_dict = dict()
for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train_binary)
        score = clf.score(X_test, y_test_binary)
        score_dict[name] = score
        print(name)
        print(classification_report(clf.predict(X_test), y_test_binary))
print(score_dict)

Linear SVM
              precision    recall  f1-score   support

           0       0.89      0.82      0.86       152
           1       0.78      0.86      0.82       108

    accuracy                           0.84       260
   macro avg       0.83      0.84      0.84       260
weighted avg       0.84      0.84      0.84       260

RBF SVM
              precision    recall  f1-score   support

           0       0.94      0.92      0.93       142
           1       0.91      0.92      0.92       118

    accuracy                           0.92       260
   macro avg       0.92      0.92      0.92       260
weighted avg       0.92      0.92      0.92       260

Gaussian Process
              precision    recall  f1-score   support

           0       0.91      0.91      0.91       141
           1       0.89      0.90      0.90       119

    accuracy                           0.90       260
   macro avg       0.90      0.90      0.90       260
weighted avg       0.90      0.90     

In [21]:
#yet another logistic regression 
logReg = LogisticRegression(random_state=0, max_iter = 50000)
logReg.fit(X_train, y_train)

test_training= logReg.score(X_train, y_train)
test_testing = logReg.score(X_test, y_test)

print("training data score", test_training, "\ntesting data score", test_testing)

training data score 0.6535129932627527 
testing data score 0.6192307692307693


In [22]:
#multiclass logistic regression
multiclassLogReg = LogisticRegression(multi_class='ovr')
multiclassLogReg.fit(X_train, y_train)

multi_test_training= multiclassLogReg.score(X_train, y_train)
multi_test_testing = multiclassLogReg.score(X_test, y_test)

print("training data score", multi_test_training, "\ntesting data score", multi_test_testing)

training data score 0.6323387872954764 
testing data score 0.6076923076923076


In [23]:
# Vanilla SVM, https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
y = np.array([1, 1, 2, 2])
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X, y)
print(clf.predict([[-0.8, -1]]))

[1]


In [24]:
# Multinomial Naive Bayes
rng = np.random.RandomState(1)
X = rng.randint(5, size=(6, 100))
y = np.array([1, 2, 3, 4, 5, 6])
clf = MultinomialNB()
clf.fit(X, y)
print(clf.predict(X[2:3]))

[3]


In [25]:
# Gaussian NB
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
Y = np.array([1, 1, 1, 2, 2, 2])
clf = GaussianNB()
clf.fit(X, Y)
print(clf.predict([[-0.8, -1]]))

clf_pf = GaussianNB()
clf_pf.partial_fit(X, Y, np.unique(Y))
print(clf_pf.predict([[-0.8, -1]]))

[1]
[1]


In [26]:
# Logistic Regression, https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0, max_iter = 50000).fit(X, y)
print(clf.predict(X[:2, :]))

print(clf.predict_proba(X[:2, :]))
print(clf.score(X, y))

[0 0]
[[9.81584563e-01 1.84154230e-02 1.45015560e-08]
 [9.71337814e-01 2.86621561e-02 3.01918305e-08]]
0.9733333333333334


In [27]:
# Linear SVM for large datasets, https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC
X, y = datasets.load_digits(n_class=9, return_X_y=True)
data = X / 16
feature_map_nystroem = Nystroem(gamma=.2,
                                random_state=1,
                                n_components=300)
data_transformed = feature_map_nystroem.fit_transform(data)

clf = make_pipeline(StandardScaler(),
                  LinearSVC(random_state=0, tol=1e-5, max_iter = 50000))
clf.fit(data_transformed, y)
clf.score(data_transformed, y)

1.0

In [28]:
# Random Forest Classification, https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
X, y = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)
print(clf.predict([[0, 0, 0, 0]]))

[1]


In [29]:
# Decision Tree Classifier, https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
iris = load_iris()
cross_val_score(clf, iris.data, iris.target, cv=10)

array([1.        , 0.93333333, 1.        , 0.93333333, 0.93333333,
       0.86666667, 0.93333333, 1.        , 1.        , 1.        ])

In [30]:
# Bagging Classifier, https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html
X, y = make_classification(n_samples=100, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
clf = BaggingClassifier(base_estimator=SVC(),
                        n_estimators=10, random_state=0).fit(X, y)
clf.predict([[0, 0, 0, 0]])

array([1])

In [31]:
# Gradient Boosting Classifier, https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
      max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.913

In [32]:
# MLP Classifier, https://scikit-learn.org/stable/modules/neural_networks_supervised.html
X = [[0., 0.], [1., 1.]]
y = [0, 1]
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)

clf.fit(X, y)
clf.predict([[2., 2.], [-1., -2.]])

array([1, 0])

## Rebalancing the Dataset

In [32]:
oversample = SMOTE(kind='svm')
def f(i):
    if i == 11:
        return 0
    else:
        return 1
    
a = np.where(label == 11)
data_pos = np.delete(data, a, axis=0)
label_pos = np.delete(label, a)
print(data_pos.shape)
print(label_pos.shape)

X_train, X_test, y_train, y_test = train_test_split(data_pos, label_pos, test_size=0.2, random_state=42, stratify=label_pos)

X_train_rebal, y_train_rebal = oversample.fit_resample(X_train, y_train)
print(Counter(y_train_rebal.tolist()))

TypeError: __init__() got an unexpected keyword argument 'kind'

## Multiclass classification, positive labels only, unbalanced

In [30]:
names = ["Linear SVM", "RBF SVM", "Logistic Regression",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "Bagging Classifier"]
         
classifiers = [
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    # GaussianProcessClassifier(1.0 * RBF(1.0), n_jobs=-1),
    LogisticRegression(random_state=0, max_iter = 5000),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier(alpha=1, max_iter=5000),
    AdaBoostClassifier(),
    GaussianNB(),
    BaggingClassifier(base_estimator=SVC(),
                        n_estimators=10, random_state=0)]

score_dict = dict()
for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        score_dict[name] = score
        print(name)
        print(classification_report(clf.predict(X_test), y_test, zero_division=0))
print(score_dict)

Linear SVM
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       1.00      0.23      0.38       120
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0

    accuracy                           0.23       120
   macro avg       0.10      0.02      0.04       120
weighted avg       1.00      0.23      0.38       120

RBF SVM
              precision    recall  f1-score   support

           1       0.38      0.62      0.48         8
           2       0.89      0.31      0.46        80
           3       0.00      0.00      0.00         0
     

## Multiclass classification, positive labels only, balanced

In [31]:
names = ["Linear SVM", "RBF SVM", "Logistic Regression",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "Bagging Classifier"]
         
classifiers = [
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    # GaussianProcessClassifier(1.0 * RBF(1.0), n_jobs=-1),
    LogisticRegression(random_state=0, max_iter = 5000),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier(alpha=1, max_iter=5000),
    AdaBoostClassifier(),
    GaussianNB(),
    BaggingClassifier(base_estimator=SVC(),
                        n_estimators=10, random_state=0)]

score_dict = dict()
for name, clf in zip(names, classifiers):
        clf.fit(X_train_rebal, y_train_rebal)
        score = clf.score(X_test, y_test)
        score_dict[name] = score
        print(name)
        print(classification_report(clf.predict(X_test), y_test, zero_division=0))
print(score_dict)

Linear SVM
              precision    recall  f1-score   support

           1       0.46      0.43      0.44        14
           2       0.43      0.67      0.52        18
           3       0.50      0.12      0.20        24
           4       0.18      0.80      0.30         5
           5       0.36      0.44      0.40         9
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         2
           8       0.42      0.33      0.37        15
           9       0.50      0.05      0.09        21
          10       0.23      0.33      0.27         9

    accuracy                           0.32       120
   macro avg       0.31      0.32      0.26       120
weighted avg       0.41      0.32      0.29       120

RBF SVM
              precision    recall  f1-score   support

           1       0.38      0.38      0.38        13
           2       0.64      0.34      0.44        53
           3       0.00      0.00      0.00         3
     

### Using data pipeline for binary logistic regression

In [33]:
#Using data pipeline

data = DataLoader('../data/EMNLP2020.csv').load()

# Load preprocessor
lp = LemmatizerPreprocessor()

NameError: name 'DataLoader' is not defined

In [ ]:
w2v = Word2Vec(path='../data/glove.6B/glove.6B.200d.txt')

In [ ]:
# binary case
X, y = w2v.featurize(data, lp, mode='binary')



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
#logistic regression with binary labels

logReg = LogisticRegression(random_state=0, max_iter = 50000)
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)
logReg.fit(X_train, y_train)

test_training= logReg.score(X_train, y_train)
test_testing = logReg.score(X_test, y_test)

print("training data score", test_training, "\ntesting data score", test_testing)